# **Task 5: Inductive Biases of Models: Locality Biases**</br>
The aim of this task is to understand the model’s (EfficientNet-B3) reliance on local and global information. For this, use a standard
image classification dataset; you can again use CIFAR-10 which is manageable in size but complex enough to
demonstrate the strengths and weaknesses of each model type in handling local and global features. Do the
following:
1. Localized Noise Injection: Here we investigate how does the introduction of noise to a localized region of
the image affect the performance of each model?
Create a subset of CIFAR-10 by introducing random noise (e.g., Gaussian noise) to a small, localized region
(e.g., a randomly selected 32x32 pixel patch) of images selected from CIFAR-10 dataset. To isolate the effects
of localized noise, the same noise pattern should be applied to different images at the same location to ensure
consistency across tests for all models. In this way, you are simulating practical scenarios where part of an
image might be obscured or corrupted.
Measure how the introduction of this localized noise affects the accuracy of your models by comparing the
performance on noisy images to that on clean images.
2. Global Style Changes: Here we investigate how do global alterations in the style or color of the entire
image influence the classification accuracy of each neural network architecture?
4
Create a subset of CIFAR-10 by applying a style transfer or color shift across the entire image. Use a style
transfer model (like a pre-trained VGG network in combination with a style image) to modify the style of the
entire image.
Measure how the introduction of global style transfer impacts the accuracy of your models by comparing the
performance on pre vs post style transfer images.
3. Scrambled Images: Here we investigate how do random permutations in image patches influence the classification
accuracy of each neural network architecture?
Create another subset of CIFAR-10 by dividing images into patches and randomly scrambling them. The task
for the models will be to classify images based on the global arrangement of these patches, which requires
understanding the overall structure rather than local cues.
Evaluate each model’s effectiveness in dealing with scrambled images by comparing their accuracies

In [39]:
pip install timm

In [40]:
import tensorflow as tf
tf.test.gpu_device_name()
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Defining the Efficient Net Model**

In [41]:
import timm
import torch
import torch.nn as nn


class EfficientNetB3Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(EfficientNetB3Model, self).__init__()
        self.enetb3 = timm.create_model('efficientnet_b3', pretrained=pretrained)
        self.enetb3.classifier = nn.Linear(self.enetb3.classifier.in_features, num_classes)



    def forward(self, x):
        x = self.enetb3(x)
        return x

def load_efficientnetb3_model(num_classes, device,task):
    model = EfficientNetB3Model(num_classes)
    if task!='nopath':
        model.load_state_dict(torch.load(f'fine_tuned_enetb3_{task}.pth'))
    model = model.to(device)
    return model
print("sanity check")

sanity check


# **Verifying the Model**

In [42]:
import torch
# from efficientnet_b3_model import load_efficientnetb3_model

def verify_efficientnetb3_model():
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    print(f"Using device: {device}")

    num_classes = 10

    model = load_efficientnetb3_model(num_classes, device, task='nopath')
    print(f"Model loaded successfully. Number of classes: {num_classes}")

    print("\nModel Architecture:")
    print(model)

    batch_size = 1
    dummy_input = torch.randn(batch_size, 3, 112, 112).to(device)
    print(f"\nDummy input shape: {dummy_input.shape}")

    try:
        with torch.no_grad():
            output = model(dummy_input)
        print("Forward pass successful!")
        print(f"Output shape: {output.shape}")

        expected_shape = (batch_size, num_classes)
        assert output.shape == expected_shape, f"Expected output shape {expected_shape}, but got {output.shape}"
        print("Output shape is correct.")

        if device.type == 'cuda':
          torch.cuda.empty_cache()
        elif device.type == 'mps':
          torch.mps.empty_cache()

    except Exception as e:
        print(f"Error during forward pass: {str(e)}")
        return

    print("\nModel verification completed successfully!")
print("sanity check")

sanity check


In [43]:
verify_efficientnetb3_model()

Using device: cpu
Model loaded successfully. Number of classes: 10

Model Architecture:
EfficientNetB3Model(
  (enetb3): EfficientNet(
    (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (bn1): BatchNormAct2d(
            40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(10, 40, ker

# **Loading the CIFAR 10 Dataset**

In [44]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import os
import torch
from torch.utils.data import Dataset
from PIL import Image



class CustomCIFAR:
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.categories = sorted(os.listdir(os.path.join(root_dir)))
        self.images = []
        self.labels = []

        for category in self.categories:
            category_dir = os.path.join(root_dir, category)
            for image_file in os.listdir(category_dir):
                image_path = os.path.join(category_dir, image_file)
                self.images.append(image_path)
                self.labels.append(self.categories.index(category))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = self.images[index]
        label = self.labels[index]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image, label


def get_data_loaders_cifar10(batch_size, task):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])




    # Load CIFAR-10 dataset

    train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

    # # Get the list of class names
    # class_names = train_dataset.classes  # This will return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    # train_dataset.targets=class_names[train_dataset.targets]

    if (task=='og'):
      test_dataset= CustomCIFAR(root_dir='/content/drive/MyDrive/Advanced_ML/PA1/task5_original_images', transform=transform)
    if (task=='local'):
      test_dataset= CustomCIFAR(root_dir='/content/drive/MyDrive/Advanced_ML/PA1/task5_noisy_images', transform=transform)
    if (task=='style'):
      test_dataset= CustomCIFAR(root_dir='/content/drive/MyDrive/Advanced_ML/PA1/task5_stylized_images', transform=transform)
    if (task=='scrambled'):
      test_dataset= CustomCIFAR(root_dir='/content/drive/MyDrive/Advanced_ML/PA1/task5_scrambled_images', transform=transform)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


    # train_labels = train_dataset.targets
    # print("Training Labels:", train_labels[:10])  # Print the first 10 labels for example

    # for i, (images, labels) in enumerate(test_loader):
    #   for j, label in enumerate(labels):
    #       print(f"Image {i*len(labels) + j + 1}, Label: {label.item()}")
    #   if i == 0:  # Stop after 10 batches (640 images)
    #       break
    return train_loader, test_loader, 10

In [45]:
get_data_loaders_cifar10(batch_size=64, task='scrambled')

Files already downloaded and verified


(<torch.utils.data.dataloader.DataLoader at 0x7e728c3e74c0>,
 10)

# **Fine Tuning the Model to the CIFAR Dataset**

In [46]:
import timm
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.amp import autocast, GradScaler
import time
# from efficientnet_b3_model import load_efficientnetb3_model
# from data_cifar10 import get_data_loaders_cifar10

def train_model(model, train_loader, criterion, optimizer, device, num_epochs=5):
    model.train()
    scaler = GradScaler()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        start_time = time.time()


        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            with autocast(device_type=device.type):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if (batch_idx + 1) % 20 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}] completed in {time.time() - start_time:.2f} seconds. "
              f"Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

print ("sanity check")

sanity check


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader, test_loader, num_classes = get_data_loaders_cifar10(64,'og')
model = load_efficientnetb3_model(num_classes, device, task='nopath')

for name, param in model.named_parameters():
    print(name, param.requires_grad)

for name, param in model.named_parameters():
    if "some_specific_layer" in name:
        param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

train_model(model, train_loader, criterion, optimizer, device, num_epochs=2)

torch.save(model.state_dict(), 'fine_tuned_enetb3_task5.pth')


print("hogya")

Files already downloaded and verified
enetb3.conv_stem.weight True
enetb3.bn1.weight True
enetb3.bn1.bias True
enetb3.blocks.0.0.conv_dw.weight True
enetb3.blocks.0.0.bn1.weight True
enetb3.blocks.0.0.bn1.bias True
enetb3.blocks.0.0.se.conv_reduce.weight True
enetb3.blocks.0.0.se.conv_reduce.bias True
enetb3.blocks.0.0.se.conv_expand.weight True
enetb3.blocks.0.0.se.conv_expand.bias True
enetb3.blocks.0.0.conv_pw.weight True
enetb3.blocks.0.0.bn2.weight True
enetb3.blocks.0.0.bn2.bias True
enetb3.blocks.0.1.conv_dw.weight True
enetb3.blocks.0.1.bn1.weight True
enetb3.blocks.0.1.bn1.bias True
enetb3.blocks.0.1.se.conv_reduce.weight True
enetb3.blocks.0.1.se.conv_reduce.bias True
enetb3.blocks.0.1.se.conv_expand.weight True
enetb3.blocks.0.1.se.conv_expand.bias True
enetb3.blocks.0.1.conv_pw.weight True
enetb3.blocks.0.1.bn2.weight True
enetb3.blocks.0.1.bn2.bias True
enetb3.blocks.1.0.conv_pw.weight True
enetb3.blocks.1.0.bn1.weight True
enetb3.blocks.1.0.bn1.bias True
enetb3.blocks.1.0

# **Evaluation**

In [48]:
import torch
# from efficientnet_b3_model import load_efficientnetb3_model
# from data_cifar10_task5 import get_data_loaders_cifar10
from sklearn.metrics import confusion_matrix
import numpy as np

def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    all_labels=[]
    all_predicted=[]

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predicted.extend(predicted.cpu().numpy())

    conf_matrix=confusion_matrix(all_labels,all_predicted)
    classwise_accuracies=np.zeros((10,1))
    for i in range(10):
        total_class_labels=0
        for j in range(10):
            total_class_labels += conf_matrix[i,j]
        classwise_accuracies[i,0]=conf_matrix[i,i]/total_class_labels
    accuracy = 100 * correct / total

    print("Confusion Matrix")
    print(conf_matrix)
    print(f"Accuracy on CIFAR-10 test set: {accuracy:.2f}%")
    print("Classwise Accuracies:")
    print(classwise_accuracies)
    return accuracy
print("sanity check")


sanity check


In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load model
model = load_efficientnetb3_model(num_classes, device,task='task5')
print("sanity check")

<ipython-input-41-573a8c121e32>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'fine_tuned_enetb3_{task}.pth'))


sanity check


# **Evaluation on Original Dataset**

In [50]:
train_loader, test_loader, num_classes = get_data_loaders_cifar10(64,task='og')
print("Original Dataset evaluation")
og_acc=evaluate_model(model, test_loader, device)

Files already downloaded and verified
Original Dataset evaluation
Confusion Matrix
[[50  0  0  0  0  0  0  0  0  0]
 [ 0 49  0  0  0  0  0  0  0  1]
 [ 0  0 50  0  0  0  0  0  0  0]
 [ 0  0  0 50  0  0  0  0  0  0]
 [ 0  0  0  0 50  0  0  0  0  0]
 [ 0  0  0  0  0 50  0  0  0  0]
 [ 0  0  0  0  0  0 50  0  0  0]
 [ 0  0  0  0  0  0  0 50  0  0]
 [ 0  1  0  0  0  0  0  0 49  0]
 [ 0  0  0  0  0  0  0  0  0 50]]
Accuracy on CIFAR-10 test set: 99.60%
Classwise Accuracies:
[[1.  ]
 [0.98]
 [1.  ]
 [1.  ]
 [1.  ]
 [1.  ]
 [1.  ]
 [1.  ]
 [0.98]
 [1.  ]]


In [51]:
print("Accuracy on Original Dataset: ", og_acc/100)

Accuracy on Original Dataset:  0.996


# **Localized Noise Injection Evaluation**

In [52]:
# Load CIFAR-10 dataset
train_loader, test_loader1, num_classes = get_data_loaders_cifar10(64,task='local')
print("Localized Noise Injection evaluation")
local_acc=evaluate_model(model, test_loader1, device)

Files already downloaded and verified
Localized Noise Injection evaluation


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Confusion Matrix
[[936   7  47  23  11   8   3   2  11   2]
 [  5 240   2  14   1   1   0   0   2   1]
 [  7   0  30  11   1   0   1   0   0   0]
 [  4   0   3  43   0   0   0   0   0   0]
 [  5   0  26   5   9   2   2   1   0   0]
 [  2   0   7  26   1  11   1   0   2   0]
 [  4   0  15  12   1   0  18   0   0   0]
 [  7   0   5  17   3   4   0  14   0   0]
 [ 20   2   6   1   0   0   1   0  20   0]
 [ 10  20   2   8   0   0   0   0   0  10]]
Accuracy on CIFAR-10 test set: 77.56%
Classwise Accuracies:
[[0.89142857]
 [0.90225564]
 [0.6       ]
 [0.86      ]
 [0.18      ]
 [0.22      ]
 [0.36      ]
 [0.28      ]
 [0.4       ]
 [0.2       ]]


In [53]:
print("Accuracy on Noisy Dataset: ",local_acc/100)

Accuracy on Noisy Dataset:  0.7756410256410257


# **Global Style Changes Evaluation**

In [54]:
train_loader, test_loader2, num_classes = get_data_loaders_cifar10(64,task='style')
print("Global Style Changes evaluation")
style_acc=evaluate_model(model, test_loader2, device)

Files already downloaded and verified
Global Style Changes evaluation


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Confusion Matrix
[[ 6  0  7  3  6 33  3  0  1  3]
 [ 5  0 11  6  2 23  2  0  0  1]
 [ 1  0 10  3  1 34  0  1  0  0]
 [ 2  0 11  2  1 29  3  1  1  0]
 [ 3  0  9  6  1 28  1  0  2  0]
 [ 2  0  9  2  7 27  1  0  1  1]
 [ 2  0  8  6  3 27  2  0  1  1]
 [ 2  0  6  6  3 30  1  1  1  0]
 [ 4  0  6  8  1 28  0  1  2  0]
 [ 3  0  8  6  4 28  1  0  0  0]]
Accuracy on CIFAR-10 test set: 9.96%
Classwise Accuracies:
[[0.09677419]
 [0.        ]
 [0.2       ]
 [0.04      ]
 [0.02      ]
 [0.54      ]
 [0.04      ]
 [0.02      ]
 [0.04      ]
 [0.        ]]


In [55]:
print("Accuracy on Stylized Dataset: ",style_acc/100)

Accuracy on Stylized Dataset:  0.099609375


# **Scrambled Images Evaluation**

In [56]:
train_loader, test_loader3, num_classes = get_data_loaders_cifar10(64,task='scrambled')
print("Scrambled Images evaluation")
scram_acc=evaluate_model(model, test_loader3, device)

Files already downloaded and verified
Scrambled Images evaluation


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Confusion Matrix
[[34  0  2  7  0  0  0  0  1  6]
 [ 5  6  2  4  0  0  0  0  1 32]
 [ 5  0 18 24  1  0  0  0  0  2]
 [ 1  0  0 43  1  0  0  1  0  4]
 [ 0  0  6 14 21  0  1  2  1  5]
 [ 3  0  0 37  5  5  0  0  0  0]
 [ 2  0  8 26  4  0 10  0  0  0]
 [ 2  0  3 24  6  0  0 11  0  4]
 [12  1  3 16  0  0  0  0  6 12]
 [ 2  0  0  2  0  0  0  0  0 46]]
Accuracy on CIFAR-10 test set: 40.00%
Classwise Accuracies:
[[0.68]
 [0.12]
 [0.36]
 [0.86]
 [0.42]
 [0.1 ]
 [0.2 ]
 [0.22]
 [0.12]
 [0.92]]


In [57]:
print("Accuracy on Scrambled Dataset: ",scram_acc/100)

Accuracy on Scrambled Dataset:  0.4
